<a href="https://colab.research.google.com/github/vanderbilt-data-science/grant-proposal-generation/blob/main/immunology_vectorstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain
!pip install -q openai
!pip install -q gradio
!pip install -q pypdf
!pip install -q deeplake
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 858.2/858.2 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.5 MB/s eta 0:

In [2]:
import openai
import random
import time
import gradio as gr
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader

In [3]:
# add API key for the session

##NOTE: YOU WILL NEED TO SET THE API KEY BEFORE CREATING THE DATASTORE
from getpass import getpass
openai_api_key = getpass()

os.environ["OPENAI_API_KEY"] = openai_api_key


··········


In [4]:
# Access files directly from GitHub
!mkdir "papers"

# This doesn't work anymore
# INSTRUCTIONS: GO TO THE BELOW LINKS, DOWNLOAD, AND UPLOAD TO DIRECTORY "PAPERS"
!wget "https://github.com/vanderbilt-data-science/grant-proposal-generation/raw/main/literature/antigenic_mapping.pdf" -P papers
!wget "https://github.com/vanderbilt-data-science/grant-proposal-generation/raw/main/literature/neutralizing_antibodies.pdf" -P papers
!wget "https://github.com/vanderbilt-data-science/grant-proposal-generation/raw/main/literature/philosophy_of_immunology.pdf" -P papers

--2023-05-19 16:17:59--  https://github.com/vanderbilt-data-science/grant-proposal-generation/raw/main/literature/antigenic_mapping.pdf
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vanderbilt-data-science/grant-proposal-generation/main/literature/antigenic_mapping.pdf [following]
--2023-05-19 16:17:59--  https://raw.githubusercontent.com/vanderbilt-data-science/grant-proposal-generation/main/literature/antigenic_mapping.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5469040 (5.2M) [application/octet-stream]
Saving to: ‘papers/antigenic_mapping.pdf’

antigenic_mapping.p 100%[=========

In [5]:
#create vector store 

root_dir = 'papers'


pdf_dir = 'papers'
pdf_loader = PyPDFDirectoryLoader(pdf_dir)
pdf_docs = pdf_loader.load_and_split()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pdf_docs)
print(f"{len(texts)}")

180


In [ ]:
embeddings = OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, embedding_ctx_length=8191, openai_api_key=None, openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=6, request_timeout=None, headers=None)

In [ ]:
db = DeepLake.from_documents(texts, dataset_path="./documentation_db", embedding=embeddings, overwrite=True)

./documentation_db loaded successfully.


Evaluating ingest: 100%|██████████| 1/1 [00:06<00:00


Dataset(path='./documentation_db', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (180, 1536)  float32   None   
    ids      text     (180, 1)      str     None   
 metadata    json     (180, 1)      str     None   
   text      text     (180, 1)      str     None   


In [ ]:
def respond(message, chat_history):

    # Get embeddings
    embeddings = OpenAIEmbeddings()
    
    #Connect to existing vectorstore
    db = DeepLake(dataset_path="./documentation_db", embedding_function=embeddings, read_only=True)
    #Set retriever settings
    retriever = db.as_retriever(search_kwargs={"distance_metric":'cos',
                                               #"fetch_k":5,
                                               "maximal_marginal_relevance":True,
                                               "reduce_k_below_max_tokens":True})
                                               #"k":5})

    if len(chat_history) != 0: 
        chat_history = [(chat_history[0][0], chat_history[0][1])]

    # Create ChatOpenAI and ConversationalRetrievalChain
    model = ChatOpenAI(model_name="gpt-3.5-turbo") #they changed this in the latest version ##annoying (model == model_name)
    qa = ConversationalRetrievalChain.from_llm(model, retriever)

    bot_message = qa({"question": message, "chat_history": chat_history})
    chat_history = [(message, bot_message["answer"])]
    time.sleep(1)
    return "", chat_history

In [ ]:
respond("Write a paragraph detailing recent advancements in immunology research.", []) #context length issues with 3.5-turbo #implement different memory system

./documentation_db loaded successfully.



Deep Lake Dataset in ./documentation_db already exists, loading from the storage
Dataset(path='./documentation_db', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (180, 1536)  float32   None   
    ids      text     (180, 1)      str     None   
 metadata    json     (180, 1)      str     None   
   text      text     (180, 1)      str     None   


('',
 [('Write a paragraph detailing recent advancements in immunology research.',
   "In recent years, advancements in immunology research have been significant and promising. The role of the immune system in cancer development and treatment has been a major focus, leading to the development of cancer immunotherapies that have shown unprecedented results in previously incurable cancers. Immune checkpoint inhibitors have particularly been explored and have led to the Nobel Prize in Physiology/Medicine being awarded to James P. Allison and Tasuku Honjo for their discovery of cancer therapy by inhibition of negative immune regulation. Additionally, research on the microbiome and its interactions with the immune system has shed light on the role of gut bacteria in various diseases and disorders. Overall, these advancements in immunology research offer new avenues for therapeutic interventions and a deeper understanding of the immune system's role in health and disease.")])

# Other memory options

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
#Set retriever settings
retriever = db.as_retriever(search_kwargs={"distance_metric":'cos',
                                            "fetch_k":5,
                                            "maximal_marginal_relevance":True,
                                            "k":5})

model = ChatOpenAI(model_name="gpt-3.5-turbo") #they changed this in the latest version ##annoying (model == model_name)
qa = ConversationalRetrievalChain.from_llm(model, retriever, memory=memory)

In [ ]:
# EXAMPLE ask about immunology
chat_history = []
query = "what are major areas of immunology research?"
result = qa({"question": query, "chat_history": chat_history})

In [ ]:
result

{'question': 'what are major areas of immunology research?',
 'chat_history': [HumanMessage(content='what are major areas of immunology research?', additional_kwargs={}, example=False),
  AIMessage(content='Immunology research covers a wide range of topics, including the defense of the organism against pathogens, cancer, infectious diseases, vaccination, transplantation, autoimmune diseases, chronic inflammatory diseases, metabolic diseases, development, aging, repair and regeneration, and host-microbiota interactions, among others. Immunology is also central for our understanding of autoimmunity, allergies, and transplant rejection. Additionally, immunology plays a major role in the development and use of immunotherapies for cancer and other diseases.', additional_kwargs={}, example=False)],
 'answer': 'Immunology research covers a wide range of topics, including the defense of the organism against pathogens, cancer, infectious diseases, vaccination, transplantation, autoimmune diseas

In [ ]:
# EXAMPLE try a grant generation
chat_history = []
query = "In a confident and excited voice, write an introduction for a grant for research on cancer immunology."
result = qa({"question": query, "chat_history": chat_history})

In [ ]:
result

{'question': 'In a confident and excited voice, write an introduction for a grant for research on cancer immunology.',
 'chat_history': [HumanMessage(content='what are major areas of immunology research?', additional_kwargs={}, example=False),
  AIMessage(content='Immunology research covers a wide range of topics, including the defense of the organism against pathogens, cancer, infectious diseases, vaccination, transplantation, autoimmune diseases, chronic inflammatory diseases, metabolic diseases, development, aging, repair and regeneration, and host-microbiota interactions, among others. Immunology is also central for our understanding of autoimmunity, allergies, and transplant rejection. Additionally, immunology plays a major role in the development and use of immunotherapies for cancer and other diseases.', additional_kwargs={}, example=False),
  HumanMessage(content='In a confident and excited voice, write an introduction for a grant for research on cancer immunology.', additional

# Workflow

1. Run all of the cells above (if interested, check the responses to the EXAMPLE cells for correctness).

  a) Enter OpenAI API key in code cell #3 

  b) If the files will not automatically upload to Colab, follow their GitHub links, download them manually, and upload to Colab in the "papers" directory.

2. Insert your prompts into the cells below, following instructions for each. 

## First Step: Initialize Starting Grant

In [ ]:
query = "The following is a {section} of a grant on {topic}. {grant}"



In the below cell, enter the section name in the `section` variable, paste the example grant section in the `grant` variable, and enter the topic in the `topic` variable.

In [ ]:
section = "ENTER SECTION NAME HERE (EX: INTRODUCTION)"
grant = "PASTE GRANT HERE"
topic = "ENTER TOPIC HERE (EX: CANCER IMMUNOLOGY)"

In [ ]:
query = query.format(section = section, topic = topic, grant = grant)
print(query)

The following is a ENTER SECTION NAME HERE (EX: INTRODUCTION) of a grant on ENTER TOPIC HERE (EX: CANCER IMMUNOLOGY). PASTE GRANT HERE


In [ ]:
result = qa({"question": query, "chat_history": chat_history})
print(result)

## Second Step: Generate Grant based on example grant

In [ ]:
query = "Using the previous grant section as a template, now write a {section} of a grant on {new_topic}"

As in the first step, add in the desired grant section and grant topic into the below variables.

In [ ]:
section = "ENTER SECTION NAME HERE"
new_topic = "ENTER NEW TOPIC HERE"

In [ ]:
query = query.format(section = section, new_topic = new_topic)
print(query)

The below cell should now output the new grant section! Iterate on this using prompt engineering until you receive desired output.

In [ ]:
result = qa({"question": query, "chat_history": chat_history})
print(result)

Keep doing this with the below cells until you have an entire grant!

In [ ]:
query = "Now write the {next_section} for the grant section you just generated."
next_section = "ENTER NEXT SECTION"
query = query.format(next_section = next_section)
print(query)

In [ ]:
result = qa({"question": query, "chat_history": chat_history})
print(result)

In [ ]:
## REPEAT ABOVE UNTIL DESIRED OUTCOME ACHIEVED ##

Now let's save the output and see our entire grant!

In [ ]:
generated_grant_chat = chat_history

We will most likely need to format and edit this. But what a great start!